## Calculating the functional connectivity within the left hemisphere
##### Data: task-based fMRI (HCP-YA-S1200), emotional processing task
##### Contributor: Jiayue Yang
##### Time: August 2024

In [1]:
# import libraries and helper functions
import seaborn as sns
import matplotlib.pyplot as plt
from tfMRI_helper_functions import *

In [2]:
# define the parcellation indices that belong to each region
# frontal
frontal = np.array([12, 13, 14, 15, 16])
frontal_names = ['ctx_lh_G_front_inf-Opercular', 'ctx_lh_G_front_inf-Orbital', 'ctx_lh_G_front_inf-Triangul',
                 'ctx_lh_G_front_middle', 'ctx_lh_G_front_sup']

# anterior cingulate cortex 
ACC = np.array([6, 7])
ACC_names = ['ctx_lh_G_and_S_cingul-Ant', 'ctx_lh_G_and_S_cingul-Mid-Ant']

# insula
insula = np.array([17, 18, 48, 49, 50])
insula_names = ['ctx_lh_G_Ins_lg_and_S_cent_ins', 'ctx_lh_G_insular_short', 'ctx_lh_S_circular_insula_ant', 
                'ctx_lh_S_circular_insula_inf', 'ctx_lh_S_circular_insula_sup']

### Connectivity between all vertices

In [3]:
# concatenate all rois
hemis = 'left'
parc_atlas = 'Destrieux09_parcellation'

# create a new dataframe
all_subjects_data = {}

In [4]:
# access subject ids
subject_list = pd.read_csv(
   "subjects_647.csv",
)
all_ids = list((subject_list['Subject']))

# convert integers to string
all_ids = [str(i) for i in all_ids]

In [5]:
# iterate through all subjects
for subject_id in all_ids:
    # extract surface data
    surface_data = extract_tfMRI(subject_id)
    # frontal-ACC
    roi_time_series1 = extract_time_series(subject_id, hemis, parc_atlas, frontal, surface_data)
    # average across frontal regions
    time_series_arrays = list(roi_time_series1.values())
    stacked_arrays = np.stack(time_series_arrays, axis=0)
    frontal_time_series = np.mean(stacked_arrays, axis=0)

    roi_time_series2 = extract_time_series(subject_id, hemis, parc_atlas, ACC, surface_data)
    # average across ACC regions
    time_series_arrays = list(roi_time_series2.values())
    stacked_arrays = np.stack(time_series_arrays, axis=0)
    ACC_time_series = np.mean(stacked_arrays, axis=0)
    # generate the FC of each subject
    FC_1 = calculate_between_roi_FC(frontal_time_series, ACC_time_series)

    # frontal-insula
    roi_time_series3 = extract_time_series(subject_id, hemis, parc_atlas, insula, surface_data)
    # average across insula regions
    time_series_arrays = list(roi_time_series3.values())
    stacked_arrays = np.stack(time_series_arrays, axis=0)
    insula_time_series = np.mean(stacked_arrays, axis=0)
    # generate the FC of each subject
    FC_2 = calculate_between_roi_FC(frontal_time_series, insula_time_series)

    # ACC-insula
    FC_3 = calculate_between_roi_FC(ACC_time_series, insula_time_series)
    
    # simplify the FC matrix
    new_data = [FC_1, FC_2, FC_3]
    
    # concatenate all subject FC into the dictionary (all subjects)
    all_subjects_data[subject_id] = new_data


In [6]:
# Convert the dictionary to a DataFrame
df = pd.DataFrame(all_subjects_data).T

# Define new column labels
new_column_labels = ['frontal-ACC', 'frontal-insula', 'ACC-insula']

# Update column labels
df.columns = new_column_labels

# Add 'Subjects' column with subject names
df.insert(0, 'Subjects', df.index)

# Optionally reset the index if you want to drop the old index
df.reset_index(drop=True, inplace=True)
df.head()

,Subjects,frontal-ACC,frontal-insula,ACC-insula
0,877269,0.947870,0.920430,0.897805
1,878877,0.550691,0.158005,0.143967
2,880157,0.504856,0.378377,0.372345
3,882161,0.754626,0.811991,0.796183
4,885975,0.513672,0.433786,0.613959


In [7]:
# save FC matrix into a csv file
df.to_csv('left_hemis_FC_btw_roi.csv', index=False)